In [1]:
import csv
import numpy as np
import pandas as pd
import math
import mpu
from datetime import datetime,time,timedelta

In [2]:
#Read PlayCar Data
PlayCar=pd.read_csv("/Users/samiraziyadidegan/Documents/Carsharing_Luca/Spostamenti_Casa_Lavoro.csv")
PlayCar.head(2)

,N,DAY_START,START,DAY_END,END,LAT_HOME,LON_HOME,LAT_DEST,LON_DEST
0,1,1/1/18,8:45:00,1/1/18,17:45:00,39.223491,9.129467,39.221950,9.10241
1,2,1/1/18,7:15:00,1/1/18,16:10:00,39.210025,9.138713,39.214436,9.11989


In [3]:
#Merge Date and Time
PlayCar['Start_Datetime']=PlayCar['DAY_START'] + str(' ') + PlayCar['START']
PlayCar['End_Datetime']=PlayCar['DAY_END'] + str(' ') + PlayCar['END']
PlayCar['Start_Datetime']=pd.to_datetime(PlayCar['Start_Datetime'])
PlayCar['End_Datetime']=pd.to_datetime(PlayCar['End_Datetime'])
PlayCar.head()

,N,DAY_START,START,DAY_END,END,LAT_HOME,LON_HOME,LAT_DEST,LON_DEST,Start_Datetime,End_Datetime
0,1,1/1/18,8:45:00,1/1/18,17:45:00,39.223491,9.129467,39.221950,9.102410,2018-01-01 08:45:00,2018-01-01 17:45:00
1,2,1/1/18,7:15:00,1/1/18,16:10:00,39.210025,9.138713,39.214436,9.119890,2018-01-01 07:15:00,2018-01-01 16:10:00
2,3,1/1/18,8:15:00,1/1/18,17:14:00,39.199243,9.140444,39.215199,9.110728,2018-01-01 08:15:00,2018-01-01 17:14:00
3,4,1/1/18,7:30:00,1/1/18,16:30:00,39.219507,9.112696,39.211578,9.119600,2018-01-01 07:30:00,2018-01-01 16:30:00
4,5,1/1/18,7:45:00,1/1/18,15:40:00,39.199339,9.149289,39.211641,9.119082,2018-01-01 07:45:00,2018-01-01 15:40:00


In [4]:
#Sort the data based on both start and end time
PlayCar=PlayCar.sort_values(by=['Start_Datetime', 'End_Datetime'])
PlayCar=PlayCar.reset_index(drop=True)
PlayCar=PlayCar.reset_index()
PlayCar=PlayCar.drop(columns=["N"])
#PlayCar=PlayCar.drop(columns=['N','X_HOME','Y_HOME','X_DEST','Y_DEST'])
PlayCar=PlayCar.rename(columns={"index": "User_Code"})
PlayCar.head(10)

,User_Code,DAY_START,START,DAY_END,END,LAT_HOME,LON_HOME,LAT_DEST,LON_DEST,Start_Datetime,End_Datetime
0,0,1/1/18,0:00:00,1/1/18,10:50:00,39.208740,9.127565,39.224223,9.112245,2018-01-01 00:00:00,2018-01-01 10:50:00
1,1,1/1/18,0:15:00,1/1/18,8:00:00,39.247468,9.121087,39.222414,9.103606,2018-01-01 00:15:00,2018-01-01 08:00:00
2,2,1/1/18,0:15:00,1/1/18,8:00:00,39.219485,9.125207,39.223841,9.121661,2018-01-01 00:15:00,2018-01-01 08:00:00
3,3,1/1/18,0:15:00,1/1/18,9:00:00,39.255991,9.127585,39.248423,9.133761,2018-01-01 00:15:00,2018-01-01 09:00:00
4,4,1/1/18,0:15:00,1/1/18,9:00:00,39.221169,9.109922,39.245405,9.135553,2018-01-01 00:15:00,2018-01-01 09:00:00
5,5,1/1/18,6:00:00,1/1/18,14:50:00,39.262668,9.118115,39.231593,9.098162,2018-01-01 06:00:00,2018-01-01 14:50:00
6,6,1/1/18,6:30:00,1/1/18,15:30:00,39.226184,9.107828,39.218357,9.107444,2018-01-01 06:30:00,2018-01-01 15:30:00
7,7,1/1/18,6:30:00,1/1/18,16:27:00,39.225207,9.119728,39.221956,9.102194,2018-01-01 06:30:00,2018-01-01 16:27:00
8,8,1/1/18,6:30:00,1/1/18,16:30:00,39.215790,9.128613,39.215790,9.128613,2018-01-01 06:30:00,2018-01-01 16:30:00
9,9,1/1/18,7:00:00,1/1/18,15:00:00,39.235202,9.102810,39.223841,9.121661,2018-01-01 07:00:00,2018-01-01 15:00:00


In [5]:
Station=pd.read_csv("/Users/samiraziyadidegan/Documents/Carsharing_Luca/Stations.csv")
Station=Station.sort_values(by=['Vehicles', 'Station ID'])
Station=Station.reset_index()
Station=Station.drop(columns=['index'])
Station.head()

,Station ID,Latitude,Longitude,Vehicles
0,3,39.23932,9.12917,1
1,6,39.22105,9.11683,1
2,9,39.20749,9.13407,1
3,18,39.22830,9.12290,1
4,20,39.24465,9.12822,1


In [6]:
print(min(PlayCar.Start_Datetime))
print(max(PlayCar.End_Datetime))

2018-01-01 00:00:00
2018-01-02 06:30:00


In [7]:
Steps=[]
ts=datetime(2018,1,1,0,0,0)
while ts < datetime (2018,1,2,6,45,0):
    Steps.append(datetime(ts.year, ts.month, ts.day, ts.hour, ts.minute, ts.second))
    ts += timedelta(minutes=15)

In [8]:
i_New=0
j=0
#Critical_Distance
Distance=500
#Critivcal Time
Critical_Time=3600
Num_Station=Station.shape[0]
Speed=20
Total_Vehicle_Used=0
Parking_User_Assign = [None]
Parking_User_Location_X=[None]
Parking_User_Location_Y=[None]
Criteria22=[None]
Criteria222=[None]
Station_User_X=[None]*PlayCar.shape[0]
Station_User_Y=[None]*PlayCar.shape[0]

Booking_Result=[None]*PlayCar.shape[0]
Results=[None]*PlayCar.shape[0]
Reference=[999999]*PlayCar.shape[0]

In [9]:
for t in Steps:
    
    for i in range(PlayCar.shape[0]): 
        
        Criteria=[99999]*Num_Station
        Sorted_Criteria=[]
        
        if PlayCar.Start_Datetime[i]>=t and PlayCar.Start_Datetime[i] < t+timedelta(minutes=15):
                
            if (Station.Vehicles > 0).any():
                for n in range(Num_Station):
                    if Station.Vehicles[n]>0:
                        Criteria[n] = mpu.haversine_distance((PlayCar.LAT_HOME[i],PlayCar.LON_HOME[i]),(Station.Latitude[n],Station.Longitude[n]))
                    else:
                        Criteria[n]=99999
                Sorted_Criteria = sorted(Criteria)
                s = np.array(Criteria) 
                index = np.argsort(s)
                #print("home to work: ",Criteria )

                for m in range(Num_Station):
                    if Station.Vehicles[index[m]] >0 and Sorted_Criteria[m] <= Distance/1000:
                        Booking_Result[i]="Accept"
                        Station.Vehicles[index[m]] = Station.Vehicles[index[m]]-1
                        Total_Vehicle_Used=Total_Vehicle_Used+1
                        Results[i]="Station Vehicle is Assigned"
                        Reference[i]=index[m]
                        #print(Results[i])
                        #print("t: ", t)
                        #print("i: ", i)
                        #print("Station number: ", index[m])

                        break
                 
                    elif m==Num_Station-1:
                        if Station.Vehicles[index[0]] == 0:
                            Booking_Result[i]="Reject"
                            Results[i]="No Available Vehicle1"
                            #print(Results[i])
                            
                            break 
                            
                        elif Sorted_Criteria[0] > Distance/1000 and Sorted_Criteria[0]<99999:
                            Booking_Result[i]="Reject"
                            Results[i]="Long Walking Distance"
                            #print(Results[i])
                            break  
                                    
            elif np.all(Station.Vehicles==0):
                Booking_Result[i]="Reject"
                Results[i]="No Available Vehicle2"
                #print(Results[i])
                
        if t <=  PlayCar.End_Datetime[i] and PlayCar.End_Datetime[i]< t+timedelta(minutes=15):
            if Booking_Result[i] == "Accept":
                Station.Vehicles[Reference[i]] = Station.Vehicles[Reference[i]] + 1
                #print("Return: ",Reference[i] )
                    

/Users/samiraziyadidegan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/samiraziyadidegan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [10]:
PlayCar['Booking_Result']=Booking_Result
PlayCar['Results']=Results
PlayCar.head(1)

,User_Code,DAY_START,START,DAY_END,END,LAT_HOME,LON_HOME,LAT_DEST,LON_DEST,Start_Datetime,End_Datetime,Booking_Result,Results
0,0,1/1/18,0:00:00,1/1/18,10:50:00,39.20874,9.127565,39.224223,9.112245,2018-01-01,2018-01-01 10:50:00,Accept,Station Vehicle is Assigned


In [11]:
PlayCar.to_csv("/Users/samiraziyadidegan/Documents/Carsharing_Luca/Speed_20/Casa_Lavoro_Traditional_500.csv")
